In [ ]:
import numpy as np
import os
from Curvas import Curve
from typing import List

measurement_filename = r'P:\8 - Físicos Médicos\Roy\INTECNUS---RoyQA\Medido Synergy 2024\electrones synery 2024.asc'

calculation_filename = r''

calc_curves = import_measured_curves(measurement_filename, machine='Syn')

# queda crear nueva funcion que ingresa dosis en monaco a partir de cualquier filename y asigna valor de maquina
